# Laboratorio 1 Series de Tiempo

Sofia Garcia - 22210

Julio Garcia Salas - 22076

Joaquin Campos - 22155

In [5]:
import pandas as pd

file_path = "Estadisticas_historicas_comercializacion.xlsx"

importacion_df = pd.read_excel(file_path, sheet_name="IMPORTACION", header=6, nrows=289)
consumo_df = pd.read_excel(file_path, sheet_name="CONSUMO", header=6, nrows=301)

#consumo_df.head()
importacion_df.head()


,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Naftas,Petcoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,MTBE,Orimulsión,Petróleo reconstituido,Total importación
0,2001-01-01,0.0,27748.99,214581.84,0.0,0.0,0.0,0.0,194065.738095,820.0,...,0.0,0.0,0.0,0.0,0.0,566101.99,8402,0.0,715344,2.312639e+06
1,2001-02-01,0.0,7503.57,294609.00,0.0,0.0,0.0,0.0,170703.380952,3054.0,...,0.0,0.0,0.0,0.0,0.0,489525.80,0,0.0,370166,1.769209e+06
2,2001-03-01,0.0,26304.32,315263.80,0.0,0.0,0.0,0.0,161837.371429,677.0,...,0.0,0.0,0.0,0.0,0.0,575559.68,0,0.0,360530,1.945770e+06
3,2001-04-01,0.0,7885.89,205653.00,0.0,0.0,0.0,0.0,163048.642857,3399.0,...,0.0,0.0,0.0,0.0,0.0,437745.42,8184,0.0,359527,1.623638e+06
4,2001-05-01,0.0,8443.16,278371.30,0.0,0.0,0.0,0.0,171518.861905,585.0,...,0.0,0.0,0.0,0.0,0.0,552609.13,12680,0.0,723346,2.262727e+06


DF que deseamos trabajar

In [ ]:
cols = ["Fecha", "Gasolina regular", "Gasolina superior", "Diesel alto azufre"]

importacion_df = importacion_df[cols].copy()
importacion_df["Fecha"] = pd.to_datetime(importacion_df["Fecha"])
importacion_df = importacion_df.sort_values("Fecha").reset_index(drop=True)

consumo_df = consumo_df[cols].copy()
consumo_df["Fecha"] = pd.to_datetime(consumo_df["Fecha"])
consumo_df = consumo_df.sort_values("Fecha").reset_index(drop=True)

importacion_df.head()
importacion_df.to_csv("importaciones_combustibles.csv", index=False)
consumo_df.to_csv("consumos_combustibles.csv", index=False)




,Fecha,Gasolina regular,Gasolina superior,Diesel alto azufre
0,2001-01-01,177776.50,373963.96,566101.99
1,2001-02-01,123115.99,243091.07,489525.80
2,2001-03-01,161726.42,312084.38,575559.68
3,2001-04-01,127338.74,285054.89,437745.42
4,2001-05-01,168730.19,300913.67,552609.13
